# I - Naive Bayes

## 1) With the help of pandas tools, identify the volume and the dimension of the data.


In [1]:
import pandas as pd

In [33]:
# import csv file as a dataframe object
data = pd.read_csv("reviews_by_course.csv")

# first approach
nb_samples, nb_fields = data.shape
print(f"{nb_samples} records of {nb_fields} fields")

# print a short amount of data
data.head()

# describe dataset with basic information (number of rows, mean, std ...)
# data.describe()

# count rows per column name
# data.count()

# count rows per column name (even none values)
# data.fillna("", inplace=True)
# data.count()

140320 records of 3 fields


,CourseId,Review,Label
0,2-speed-it,BOring,1
1,2-speed-it,Bravo !,5
2,2-speed-it,Very goo,5
3,2-speed-it,"Great course - I recommend it for all, especia...",5
4,2-speed-it,One of the most useful course on IT Management!,5


As we can see, the total dimension of this dataframe is 140320 rows per 3 columns.


# 2) Target classes are the ratings, from 1 to 5. Determine the number of instances for each class. What could be the problem during the learning process ?

In [37]:
# first and naive approach : --> storing the result in a list of dataframes, filter each class and print the length.
print("first approach : ")
classes = []
for i in range(1,6):
    filter = data["Label"] == i
    classes.append(data[filter])
    print("classes",i,"has",len(data[filter]), "rows")
    
# second approach with pandas group by
print("\nsecond approach : ")
# for the whole columns
classes2 = data.groupby(["Label"]).size()
# for each columns
# classes2 = data.groupby(["Label"]).count()
classes2

first approach : 
classes 1 has 2867 rows
classes 2 has 2554 rows
classes 3 has 5923 rows
classes 4 has 22460 rows
classes 5 has 106516 rows

second approach : 


Label
1      2867
2      2554
3      5923
4     22460
5    106516
dtype: int64

As we can see, the 5th class has a huge amount of data related to it. While, other classes don't even represent 1/2 of the number of rows in the 5th class.

It can be a problem since, in order to predict a rating of 5, the model will be really accurate, but for the other classes, the model won't have enought data to correctly deal with them.

# 3) Each instance consists in a short paragraph of text, containing the review, and the course tag.

and

# 4) The volume of the data could be quite (too) large, depending on your computer. Together with your answer of Question 2), and using the functions head(), generate a meaningful subset of the data.

In [38]:
# Transforming sentence as vectors using the Bag of Word method
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data.head()["Review"]).toarray()
print(X)
Y = vectorizer.fit_transform(data.head()["CourseId"]).toarray()
print(Y)

[[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 1 0 0 1 1 1 1 0 1 2 0 1 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 1 1 0 1 1 1 1 0 1 1 0]]
[[1 1]
 [1 1]
 [1 1]
 [1 1]
 [1 1]]


Here we only tried with the first five lines.

But we could easily consider that, regarding of the number of rows and words in all those sentences, the vocabulary should be a huge number and will probably throw to a run out of memory.

Thus, we choose to only take the 200 first rows of each instance in order to reduce the size and moreover, to make the 5th class not over representated.

In [84]:
# getting the two column as vector representation
review_vectors = vectorizer.fit_transform(pd.concat([i.head(200)["Review"] for i in classes])).toarray()
courseid_vectors = vectorizer.fit_transform(pd.concat([i.head(200)["CourseId"] for i in classes])).toarray()
Y = list(pd.concat([i.head(200)["Label"] for i in classes]))

# creating a new dataframe with all the data in it
df = pd.DataFrame()
df.insert(value=pd.Series(list(review_vectors), index=range(0,len(review_vectors))), column="CountVectorReview", loc=0)
df.insert(value=pd.Series(list(courseid_vectors), index=range(0,len(courseid_vectors))), column="CountVectorCourseId", loc=1)
df.insert(value=pd.Series(Y, index=range(0,len(Y))), column="Label", loc=2)
df[199:201]

,CountVectorReview,CountVectorCourseId,Label
199,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
200,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",2


# 5) Using a naive Bayes approach, we are going to learn model whose objective is to predict the rating, given the tag and the review. What kind of probabilistic distribution should we use ?